In [1]:
%run _common.ipynb
%matplotlib inline

from jax.scipy.special import beta
import equinox as eqx
import matplotlib.pyplot as plt

from neuralconstitutive.constitutive import PowerLaw
from neuralconstitutive.ting import force_approach_scalar, force_retract_scalar


In [ ]:
constit = PowerLaw(E0=1.0, alpha = 0.2)

def force_approach_analytic(t, constit: PowerLaw, v, tip):
    a, b = tip.a(), tip.b()
    coeff = a*b*constit.E0*(v**b)*beta(1-constit.alpha, b)
    return coeff*(t**(b-constit.alpha))